# MSc in Data Analytics
# Sentiment Analysis

Assessment Task
Students are advised to review and adhere to the submission requirements documented after the assessment task.  
In this continuous assessment, You are required to identify and carry out an analysis of a large dataset gleaned from the twitter API. Instructions for accessing the data can be found here  
https://datascienceparichay.com/article/get-data-from-twitter-api-in-python-step-by-step-guide/  
https://www.toptal.com/apache/apache-spark-streaming-twitter  
OR You may use the data held here:  
https://archive.org/details/twitterstream?sort=-publicdate  
You must collect at least 1 year's tweets on a topic, this data should be stored as requested below, and you are then required to analyse any change sentiment that occurs over the time period that you have selected.  
Following your analysis, you are then required to make a time series forecast of the sentiment at 1 week, 1 month and 3 months going forward. This forecast must be displayed as a dynamic dashboard.   


In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import col
import json, os, time

In [3]:
# Create the Spark Context
# fix to manage the error 
# "Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("sba22243-step1").config("spark.sql.debug.maxToStringFields", 100).getOrCreate()
sqlContext = spark._wrapped

In [4]:
# read the directory and find the bz2 files containint the tweets
dataset_path = r'c:\tmp\twitter'

file_list = [] # this list contain the filenames of the dataset stored in the dataset_path folder
# explore the subfolder
for subdir, dirs, files in os.walk(dataset_path):
    for file in files:
        # if the file is a bz2 then get the filename
        if file.endswith('.bz2'):
            filename_bz2 = os.path.join(subdir, file)
            filename_bz2 = filename_bz2.replace('\\', '\\\\')
            file_list.append(os.path.join(subdir, file))
            

In [5]:
topic = "vaccine"

# Read a batch of files in parallel
for i in range(0, len(file_list), 10):
    sub_list = file_list[i:i+10]
    df = sqlContext.read.json(sub_list)
    print(df.count())
    df_filtered = df.filter((col('lang') == 'en') & col('text').rlike(f'(?i){topic}'))
    # moveve unused properties in order to save space on the storage
    df_filtered = df_filtered.drop('retweeted_status')
    df_filtered = df_filtered.drop('entities')
    df_filtered = df_filtered.drop('extended_entities')
    filename = f'c:\\\\tmp\\twitter-result\\{time.time()}'
    print(filename)
    df_filtered.write.mode("overwrite").json('file:///' + filename + '.json')

35344
c:\\tmp\twitter-result\1683333717.3704498
32915
c:\\tmp\twitter-result\1683333735.209237
31619
c:\\tmp\twitter-result\1683333751.088432
70297
c:\\tmp\twitter-result\1683333768.784323


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "C:\apps\spark-3.2.3-bin-hadoop3.2\python\lib\py4j-0.10.9.5-src.zip\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "C:\apps\spark-3.2.3-bin-hadoop3.2\python\lib\py4j-0.10.9.5-src.zip\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\Users\Mara\anaconda3\envs\tf\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 